In [1]:
import numpy as np
from sklearn.cluster import SpectralClustering
import skfuzzy as fuzz

# Set random seed for reproducibility
np.random.seed(42)

# Simulation parameters
num_providers = 3
num_regions = 3
min_dcs_per_provider = 2
max_dcs_per_provider = 5
num_vms_per_dc = 8
vm_processing_capability = 1500  # in MIPS
vm_num_cpus = 2
vm_ram = 4  # in Gb
vm_storage_capacity = 8  # in Gb

# Pricing information
provider_pricing = {
    'Provider 1': {
        'US': [0.020, 0.006, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'EU': [0.025, 0.006, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'AS': [0.027, 0.0066, 0.001, 0.0015, 0.002, 0.004, 0.008],
    },
    'Provider 2': {
        'US': [0.020, 0.0096, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'EU': [0.018, 0.0096, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'AS': [0.020, 0.0096, 0.001, 0.0015, 0.002, 0.004, 0.008],
    },
    'Provider 3': {
        'US': [0.0095, 0.00120, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'EU': [0.0090, 0.0096, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'AS': [0.0080, 0.0090, 0.001, 0.0015, 0.002, 0.004, 0.008],
    },
}

# Simulate data for each provider and region
provider_data = []
for provider_id in range(1, num_providers + 1):
    num_dcs = np.random.randint(min_dcs_per_provider, max_dcs_per_provider + 1)
    for region in ['US', 'EU', 'AS']:
        for dc_id in range(1, num_dcs + 1):
            vm_data = {
                'provider_id': provider_id,
                'region': region,
                'dc_id': dc_id,
                'num_vms': num_vms_per_dc,
                'vm_processing_capability': vm_processing_capability,
                'vm_num_cpus': vm_num_cpus,
                'vm_ram': vm_ram,
                'vm_storage_capacity': vm_storage_capacity,
                'cpu_price': provider_pricing[f'Provider {provider_id}'][region][0],
                'storage_price': provider_pricing[f'Provider {provider_id}'][region][1],
                'intra_dc_bw_price': provider_pricing[f'Provider {provider_id}'][region][2],
                'inter_region_bw_price': provider_pricing[f'Provider {provider_id}'][region][3],
                'intra_region_bw_price': provider_pricing[f'Provider {provider_id}'][region][4],
                'response_time_slo': np.random.uniform(150, 250),  # Simulated response time SLO
                'availability_slo': np.random.uniform(0.9, 1.0),  # Simulated availability SLO
            }
            provider_data.append(vm_data)

# Assuming 'provider_data' includes task-related features
features = [
    'cpu_price', 'storage_price', 'intra_dc_bw_price', 'inter_region_bw_price',
    'total_cost', 'task_count', 'task_size', 'response_time_slo', 'availability_slo'
]
data = np.array([[vm.get(feature, 0) for feature in features] for vm in provider_data])

# Data Identification Phase: Spectral Clustering
n_clusters = 3  # Adjust based on the data

# Use Spectral Clustering to identify clusters and correlate with SLA violations
spectral = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', n_neighbors=5)
cluster_labels = spectral.fit_predict(data)

# Identify data most likely to cause SLA violations if not replicated
# Here, we assume that clusters with higher density represent potential SLA violation areas
cluster_density = np.bincount(cluster_labels)
most_critical_clusters = np.argsort(cluster_density)[-2:]  # Select the top 2 densest clusters

critical_data_indices = []
for cluster in most_critical_clusters:
    cluster_indices = np.where(cluster_labels == cluster)[0]
    critical_data_indices.extend(cluster_indices)

# Critical data information
critical_data = data[critical_data_indices]

# Replica Placement Phase

# Integrate the four main parameters into the fuzzy inference system
# (response time, availability, cost, potential for data correlation)
response_time = np.arange(0, 1.1, 0.1)
availability = np.arange(0, 1.1, 0.1)
cost = np.arange(0, 1.1, 0.1)
data_correlation = np.arange(0, 1.1, 0.1)

# Define membership functions for new parameters
response_time_low = fuzz.trimf(response_time, [0, 0, 0.5])
response_time_high = fuzz.trimf(response_time, [0.5, 1, 1])

availability_low = fuzz.trimf(availability, [0, 0, 0.5])
availability_high = fuzz.trimf(availability, [0.5, 1, 1])

cost_low = fuzz.trimf(cost, [0, 0, 0.5])
cost_high = fuzz.trimf(cost, [0.5, 1, 1])

data_correlation_low = fuzz.trimf(data_correlation, [0, 0, 0.5])
data_correlation_high = fuzz.trimf(data_correlation, [0.5, 1, 1])

# Apply the membership functions to input and output variables
data_transfer_time_ratio_high = fuzz.trimf(response_time, [0.6, 1, 1])
data_transfer_time_ratio_medium = fuzz.trimf(response_time, [0.3, 0.5, 0.7])
data_transfer_time_ratio_low = fuzz.trimf(response_time, [0, 0, 0.3])

virtual_machine_load_high = fuzz.trimf(availability, [0.6, 1, 1])
virtual_machine_load_medium = fuzz.trimf(availability, [0.3, 0.5, 0.7])
virtual_machine_load_low = fuzz.trimf(availability, [0, 0, 0.3])

data_availability_nr = fuzz.trimf(cost, [0, 0, 0.5])
data_availability_r = fuzz.trimf(cost, [0.5, 1, 1])

provider_profit_very_low = fuzz.trimf(data_correlation, [0, 0, 0.2])
provider_profit_low = fuzz.trimf(data_correlation, [0.2, 0.4, 0.6])
provider_profit_medium = fuzz.trimf(data_correlation, [0.4, 0.6, 0.8])
provider_profit_high = fuzz.trimf(data_correlation, [0.6, 0.8, 1])
provider_profit_very_high = fuzz.trimf(data_correlation, [0.8, 1, 1])

placement_potential_very_low = fuzz.trimf(data_correlation, [0, 0, 0.2])
placement_potential_low = fuzz.trimf(data_correlation, [0.2, 0.4, 0.6])
placement_potential_medium = fuzz.trimf(data_correlation, [0.4, 0.6, 0.8])
placement_potential_high = fuzz.trimf(data_correlation, [0.6, 0.8, 1])
placement_potential_very_high = fuzz.trimf(data_correlation, [0.8, 1, 1])

# Fuzzy inference rules for new parameters
rule6 = np.fmin(np.fmin(data_transfer_time_ratio_low, virtual_machine_load_low), data_availability_nr)
rule7 = np.fmin(np.fmin(data_transfer_time_ratio_high, virtual_machine_load_high), data_availability_r)
rule8 = np.fmin(np.fmax(data_correlation_low, data_correlation_high), np.fmax(provider_profit_low, placement_potential_medium))

# Aggregate the rules for new parameters
aggregated = np.fmax(np.fmax(rule6, rule7), rule8)

# Defuzzify the aggregated result
placement_potential_crisp = fuzz.defuzz(data_correlation, aggregated, 'centroid')

# Print Results
print("Cluster Labels:", cluster_labels)
print("Critical Data Information:", critical_data)
print("Placement Potential:", placement_potential_crisp)


c:\Users\YANSILIYU\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:273: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


Cluster Labels: [2 2 0 0 0 0 1 1 0 0 0 1 2 0 0 0 1 2 0 0 0 0 2 0 0 0 1 0 0 2]
Critical Data Information: [[2.00000000e-02 6.00000000e-03 1.00000000e-03 1.50000000e-03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 2.29654299e+02
  9.18343479e-01]
 [2.00000000e-02 6.00000000e-03 1.00000000e-03 1.50000000e-03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 2.27969100e+02
  9.59685016e-01]
 [2.00000000e-02 9.60000000e-03 1.00000000e-03 1.50000000e-03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 2.28517596e+02
  9.19967378e-01]
 [1.80000000e-02 9.60000000e-03 1.00000000e-03 1.50000000e-03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 2.30839735e+02
  9.30461377e-01]
 [9.50000000e-03 1.20000000e-03 1.00000000e-03 1.50000000e-03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 2.25536141e+02
  9.42515587e-01]
 [8.00000000e-03 9.00000000e-03 1.00000000e-03 1.50000000e-03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 2.34453385e+02
  9.74732011e-01]
 [2.00000000e-02 6.00000000e-03 1.00000000e